In [21]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
def is_running_on_kaggle():
    return "KAGGLE_KERNEL_RUN_TYPE" in os.environ and os.environ["KAGGLE_KERNEL_RUN_TYPE"] == "Interactive"
DATA_PATH = '/kaggle/input/dcase-aml/' if is_running_on_kaggle() else 'data/'
print('Running on Kaggle' if is_running_on_kaggle() else 'Running locally')
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Running on Kaggle


In [22]:
# Check if CUDA is available
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cuda device


In [23]:
TRAIN_PATH = DATA_PATH + r"dev_data/dev_data/slider/train"
TEST_PATH = DATA_PATH +r"dev_data/dev_data/slider/test"

In [26]:
import librosa
import os
from torch.utils.data import Dataset
from librosa.feature import melspectrogram
import numpy as np
class AudioDataset(Dataset):
    def __init__(self, data_path, train):
        self.data_path = data_path
        if train:
            self.labels = [0 * len(os.listdir(self.data_path))]
        else:
            self.labels = [1 if el[0]=='a' else 0 for el in os.listdir(self.data_path)]
        print(self.data_path)
        self.audios, self.srs= zip(*[librosa.load(os.path.join(self.data_path, path)) for path in os.listdir(self.data_path)])
        self.mfccs = [librosa.feature.mfcc(y=audio, sr=self.srs[0]) for audio in self.audios]
        self.s= melspectrogram(y=self.audios[0], sr=self.srs[0])
        print(self.s.shape)
        self.spectrograms = [melspectrogram(y=audio, sr=self.srs[0]) for audio in self.audios]
        print(self.spectrograms[0].shape)
        self.spectrograms_db = [librosa.amplitude_to_db(spec, ref=np.min) for spec in self.spectrograms]
        
        print(self.spectrograms_db[0].shape)
    def __len__(self):
        return len(os.listdir(self.data_path))

    def __getitem__(self, idx):
        audio = self.audios[idx]
        label = self.labels[idx]
        sr = self.srs[idx]
        mfcc = self.mfccs[idx]
        spec = self.spectrograms[idx]
        spec_db = self.spectrograms_db[idx]
        return spec, spec_db, label

In [25]:
train_dataset = AudioDataset(data_path=TRAIN_PATH, train=True)
test_dataset = AudioDataset(data_path=TEST_PATH, train=False)

/kaggle/input/dcase-aml/dev_data/dev_data/slider/train
(128, 431)
(128, 431)
(128, 431)
/kaggle/input/dcase-aml/dev_data/dev_data/slider/test
(128, 431)
(128, 431)
(128, 431)


In [ ]:

train_dataset.spectrograms[0].shape